In [28]:
import collections
import json
import os

import _jsonnet

from seq2struct.utils import registry
from seq2struct import grammars
from seq2struct import datasets

In [22]:
os.chdir('..')
config = json.loads(_jsonnet.evaluate_file('configs/spider-20190205/arxiv-1906.11790v1.jsonnet'))

In [31]:
train_data = registry.construct('dataset', config['data']['train'])
val_data = registry.construct('dataset', config['data']['val'])
grammar = grammars.spider.SpiderLanguage(output_from=True)

In [89]:
def find_columns(query, include_from=False):
    result = set()
    
    queue = collections.deque([query])
    while queue:
        node = queue.popleft()
        type_info = grammar.ast_wrapper.singular_types[node['_type']]
        for field in type_info.fields:
            if not include_from and field.name == 'from':
                continue
            if field.name not in node:
                continue

            v = node[field.name]
            if isinstance(v, dict):
                queue.append(v)
            elif isinstance(v, list):
                queue.extend(v)
            
            if field.type == 'column':
                result.add(v)
    
    return result

def group_by_schema(data):
    result = collections.defaultdict(list)
    for example in data:
        result[example.schema.db_id].append(example)
    return result

def analyze(data):
    grouped = collections.defaultdict(list)
    grouped_column_usage = collections.defaultdict(list)
    for example in data:
        grouped[example.schema.db_id].append(example)
        grouped_column_usage[example.schema.db_id].append(find_columns(grammar.parse(example.code, 'train')))
    
    for db_id, column_usages in sorted(grouped_column_usage.items()):
        examples = grouped[db_id]
        schema = data.schemas[db_id]
        num_columns = len(schema.columns)
        all_columns = set(range(num_columns))
        used_columns = set.union(*column_usages)
        assert used_columns <= all_columns
        unused_columns = all_columns - used_columns
        
        # Group by query
        grouped_by_query = collections.defaultdict(list)
        for i, example in enumerate(examples):
            grouped_by_query[example.orig['query']].append(i)
        
        # For each group, see which columns were used in other queries
        all_example_indices = set(range(len(examples)))
        groups_with_misses = {}
        for query, query_group_indices in grouped_by_query.items():
            other_indices = all_example_indices - set(query_group_indices)
            other_used_columns = set.union(*(column_usages[:i] + column_usages[i+1:]))
            missing_columns = column_usages[query_group_indices[0]] - other_used_columns

            if missing_columns:
                groups_with_misses[query] = missing_columns
        
        print('{}: {} examples, {} columns, {} unused by any query; {} examples with missing columns'.format(
            db_id, len(column_usages), num_columns, len(unused_columns), len(groups_with_misses)))
        if groups_with_misses:
            print('======================')
            for query, missing_columns in groups_with_misses.items():
                print('- Query: {}'.format(query))
                print('- Questions:')
                for i in grouped_by_query[query]:
                    print('  - {}'.format(' '.join(examples[i].text)))
                print('- Missing columns: {}'.format(
                    ', '.join(
                        '{}.{}'.format(schema.columns[i].table.name, schema.columns[i].name)
                        for i in sorted(missing_columns))))



In [90]:
analyze(train_data)

academic: 181 examples, 43 columns, 27 unused by any query; 0 examples with missing columns
activity_1: 88 examples, 23 columns, 5 unused by any query; 0 examples with missing columns
aircraft: 46 examples, 29 columns, 13 unused by any query; 0 examples with missing columns
allergy_1: 98 examples, 13 columns, 0 unused by any query; 0 examples with missing columns
apartment_rentals: 80 examples, 32 columns, 9 unused by any query; 0 examples with missing columns
architecture: 17 examples, 18 columns, 4 unused by any query; 0 examples with missing columns
assets_maintenance: 31 examples, 65 columns, 33 unused by any query; 1 examples with missing columns
- Query: SELECT asset_model FROM Assets WHERE asset_id NOT IN (SELECT asset_id FROM Fault_Log)
- Questions:
  - Which assets did not incur any fault log ? List the asset model .
- Missing columns: ['fault', 'log'].['asset', 'id']
baseball_1: 82 examples, 353 columns, 321 unused by any query; 0 examples with missing columns
behavior_monito

In [38]:
train_example = next(iter(train_data))

In [41]:
find_columns(grammar.parse(train_example.code, 'train'))

{0, 10}

In [45]:
train_example.orig['query']

'SELECT count(*) FROM head WHERE age  >  56'

'department_management'